In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

In [3]:
# Add the package dependency to run kafka
spark_version = "3.5.4"
scala_version = "2.12"
# kafka_version = 

# https://stackoverflow.com/questions/58723314/pyspark-failed-to-find-data-source-kafka/58723724#58723724
# https://stackoverflow.com/questions/31841509/pyspark-exception-java-gateway-process-exited-before-sending-the-driver-its-p
os.environ["PYSPARK_SUBMIT_ARGS"] = f"--packages org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version} pyspark-shell"
print(os.environ["PYSPARK_SUBMIT_ARGS"])
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("test_kafka_read_stream")
    .getOrCreate()
)

# spark.stop()

--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4 pyspark-shell


25/01/22 20:32:27 WARN Utils: Your hostname, mackutis.local resolves to a loopback address: 127.0.0.1; using 192.168.1.147 instead (on interface en0)
25/01/22 20:32:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/kutis/.ivy2/cache
The jars for the packages stored in: /Users/kutis/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2310dae9-7005-4de4-859b-ddde5ac6c1b8;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.4/spark-sql-kafka-0-10_2.12-3.5.4.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4!spark-sql-kafka-0-10_2.12.jar (1901ms)
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-token-provider-kafka-0-10_2.12/3.5.4/spark-token-provider-kafka

In [23]:
kafka = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "page_view_events") \
    .load()

In [24]:
kafka_str = kafka.selectExpr("CAST(value as STRING)")

In [31]:
kafka_str.printSchema()

query = kafka_str.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# # Define a function to process each batch
# def process_batch(df, epoch_id):
#     df.show(truncate=False)  # Print the DataFrame to the console

# # Apply the function to each batch
# query = kafka_str.writeStream \
#     .foreachBatch(process_batch) \
#     .start()

# Await termination
query.awaitTermination()

root
 |-- value: string (nullable = true)



25/01/22 21:07:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/p8/kg48rc2s6qd3qwz7h48ckcw80000gn/T/temporary-120066ae-9bae-4df1-9be3-010a7cbc01e7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/01/22 21:07:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/01/22 21:07:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 37
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764660...|
|{"ts":17375764660...|
|{"ts":17375764660...|
+--------------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764660...|
|{"ts":17375764660...|
|{"ts":17375764660...|
|{"ts":17375764660...|
|{"ts":17375764670...|
+--------------------+

-------------------------------------------
Batch: 38
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764660...|
|{"ts":17375764670...|
+--------------------+

-------------------------------------------
Batch: 39
--------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 3
-------------------------------------------
-------------------------------------------
Batch: 40
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764680...|
|{"ts":17375764690...|
+--------------------+

+--------------------+
|               value|
+--------------------+
|{"ts":17375764680...|
|{"ts":17375764690...|
+--------------------+



-------------------------------------------
Batch: 41
-------------------------------------------
-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764690...|
|{"ts":17375764700...|
+--------------------+

+--------------------+
|               value|
+--------------------+
|{"ts":17375764690...|
|{"ts":17375764700...|
+--------------------+



-------------------------------------------
Batch: 5
-------------------------------------------
-------------------------------------------
Batch: 42
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764710...|
+--------------------+

+--------------------+
|               value|
+--------------------+
|{"ts":17375764710...|
+--------------------+

-------------------------------------------
Batch: 6
-------------------------------------------
-------------------------------------------
Batch: 43
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"ts":17375764710...|
|{"ts":17375764710...|
|{"ts":17375764720...|
+--------------------+

+--------------------+
|               value|
+--------------------+
|{"ts":17375764710...|
|{"ts":17375764710...|
|{"ts":17375764720...|
+--------------------+

-------------------------------------------
Batch: 7
-

In [37]:
query.awaitTermination()
query.stop()

spark.stop()

25/01/22 21:09:43 ERROR MicroBatchExecution: Query [id = 0ba72004-17d1-4a68-9714-e63b5ad94ede, runId = 37847c64-cc82-4500-a563-83b2fdc8ade5] terminated with error
org.apache.spark.SparkException: [INTERNAL_ERROR] The Spark SQL phase planning failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.
	at org.apache.spark.SparkException$.internalError(SparkException.scala:107)
	at org.apache.spark.sql.execution.QueryExecution$.toInternalError(QueryExecution.scala:536)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:548)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:219)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:900)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:218)
	at org.apache.spark.sql.execution.QueryExec

25/01/22 21:09:56 ERROR MicroBatchExecution: Query [id = 2515ca31-ed2a-4714-bb5f-a05b788a4356, runId = 387c1b5d-00b3-4105-9bf3-d2a6180d189b] terminated with error
org.apache.spark.SparkException: [INTERNAL_ERROR] The Spark SQL phase planning failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.
	at org.apache.spark.SparkException$.internalError(SparkException.scala:107)
	at org.apache.spark.sql.execution.QueryExecution$.toInternalError(QueryExecution.scala:536)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:548)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:219)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:900)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:218)
	at org.apache.spark.sql.execution.QueryExec